In [1]:
# import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '1'


In [2]:
import os
import shutil
import pathlib
import gc
import pandas as pd
import ttach as tta
from pathlib import Path
from PIL import Image
import numpy as np
import cv2 as cv
import random
import matplotlib.pyplot as plt
# from tqdm.auto import tqdm
from tqdm.notebook import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
import torch
from torch.utils.data import DataLoader, random_split
from torch.utils.data import Dataset

import torchvision
from torchvision import datasets
import cv2
from torch.cuda import amp

import torchvision.transforms as T
from torchvision.transforms import Compose, ToTensor, Resize
from torchvision.utils import make_grid
import optuna

import albumentations as A
from albumentations.pytorch import ToTensorV2
import segmentation_models_pytorch as smp
import timm

In [3]:
def seed_everything(seed=42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)

In [4]:
seed_everything(42)

In [5]:
CFGS = [ 
    
#     {
#         'model_name': 'Unet++',
#         'backbone': 'tf_efficientnet_b7_ns',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '../../output/nirjhar/qishentrialv2fpn512_tf_efficientnet_b7_ns_best_epochcv650lb675-00.bin',
#         'threshold': 0.62, #0.24,
#         'call_sign': "nir_01",
#         'tta': True
#     },

#     {
#         'model_name': 'Unet++',
#         'backbone': 'tf_efficientnet_b7_ns',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '../../output/nirjhar/qishentrialv512unetplus_tf_efficientnet_b7_ns_best_epochstage2cv669-00.bin',
#         'threshold': 0.62, #0.24,
#         'call_sign': "nir_02", 
#         'tta': True
#     },
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'eca_nfnet_l1',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '../../output/nirjhar/unetplusecarnf01_eca_nfnet_l1_best_epochstage2cv656-00.bin',
#         'threshold': 0.62, #0.24,
#         'call_sign': "nir_03",
#         'tta': True
#     },
#     {
#         'model_name': 'Unet',
#         'backbone': 'efficientnet-b7',
#         'img_size': [256, 256],
#         'num_classes': 1,
#         'model_pth':  '../../output/exp_s1_pl_03/pl_round1/efficientnet-b7-256-s2/checkpoint_dice.pth',
#         'threshold': 0.24, #0.24,
#         'call_sign': "roh_01",
#         'tta': True
#     },   
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'efficientnet-b7',
#         'img_size': [256, 256],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/contrail/output/exp_s1_pl_03/pl_round2/efficientnet-b7-256-s2/checkpoint_dice.pth',
#         'threshold': 0.62, #0.24,
#         'call_sign': "roh_02",
#         'tta': True
#     }, 
#     {
#         'model_name': 'Unet',
#         'backbone': 'efficientnet-b7',
#         'img_size': [256, 256],
#         'num_classes': 1,
#         'model_pth':  '../../output/exp_pre/Unet/efficientnet-b7-256-s2/checkpoint_dice.pth',
#         'threshold': 0.24, #0.24,
#         'call_sign': "roh_03", 
#         'tta': True
#     },   
#     {
#         'model_name': 'Unet',
#         'backbone': 'efficientnet-b7',
#         'img_size': [256, 256],
#         'num_classes': 1,
#         'model_pth':  '../../output/exp_pre/Unet/efficientnet-b7-256-s3/checkpoint_dice.pth',
#         'threshold': 0.24, #0.24,
#         'call_sign': "roh_04", 
#         'tta': True
#     },   
    
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'timm-resnest26d',
#         'img_size': [256, 256],
#         'num_classes': 1,
#         'model_pth':  '../../output/exp_pre/Unet/timm-resnest26d-256/checkpoint_dice.pth',
#         'threshold': 0.39, #0.24,
#         'call_sign': "roh_05", 
#         'tta': True
#     }, 
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'timm-resnest26d',
#         'img_size': [256, 256],
#         'num_classes': 1,
#         'model_pth':  '../../output/exp_pre/Unet/timm-resnest26d-256-s2/checkpoint_dice.pth',
#         'threshold': 0.39, #0.24,
#         'call_sign': "roh_06", 
#         'tta': True
#     }, 
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'timm-resnest26d',
#         'img_size': [256, 256],
#         'num_classes': 1,
#         'model_pth':  '../../output/exp_pre/Unet/timm-resnest26d-256-s3/checkpoint_dice.pth',
#         'threshold': 0.39, #0.24,
#         'call_sign': "roh_07", 
#         'tta': True
#     }, 
    
        
    {
        'model_name': 'Unet',
        'backbone': 'efficientnet-b7',
        'img_size': [256, 256],
        'num_classes': 1,
        'model_pth':  '/home/rohits/pv1/contrail/output/exp_02_s2/Unet/efficientnet-b7-256/checkpoint_dice_fold0.pth',
        'threshold': 0.39, #0.24,
        'call_sign': "roh_1_00", 
        'tta': True
    }, 
    
    
    {
        'model_name': 'Unet',
        'backbone': 'efficientnet-b7',
        'img_size': [256, 256],
        'num_classes': 1,
        'model_pth':  '/home/rohits/pv1/contrail/output/exp_02_s2/Unet/efficientnet-b7-256/checkpoint_dice_fold1.pth',
        'threshold': 0.39, #0.24,
        'call_sign': "roh_1_01", 
        'tta': True
    }, 
    
    
        
    {
        'model_name': 'Unet',
        'backbone': 'efficientnet-b7',
        'img_size': [256, 256],
        'num_classes': 1,
        'model_pth':  '/home/rohits/pv1/contrail/output/exp_02_s2/Unet/efficientnet-b7-256/checkpoint_dice_fold2.pth',
        'threshold': 0.39, #0.24,
        'call_sign': "roh_1_02", 
        'tta': True
    }, 
    
    
        
#     {
#         'model_name': 'Unet',
#         'backbone': 'efficientnet-b7',
#         'img_size': [256, 256],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/contrail/output/exp_02_s2/Unet/efficientnet-b7-256/checkpoint_dice_fold3.pth',
#         'threshold': 0.39, #0.24,
#         'call_sign': "roh_1_03", 
#         'tta': True
#     }, 
    
    
        
#     {
#         'model_name': 'Unet',
#         'backbone': 'efficientnet-b7',
#         'img_size': [256, 256],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/contrail/output/exp_02_s2/Unet/efficientnet-b7-256/checkpoint_dice_fold4.pth',
#         'threshold': 0.39, #0.24,
#         'call_sign': "roh_1_04", 
#         'tta': True
#     }, 



]

ensemple_type = "mine" #"No" #"mine"




In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [7]:
# class Dataset:
#     def __init__(self, df, transform=None):   
#         self.images = df['image']
#         self.labels = df['label']
#         self.transform =transform
        
#     def __len__(self):
#         return len(self.images)
        
#     def __getitem__(self, idx):
#         image = np.load(self.images[idx]).astype(float)
#         if  self.transform :
#             data = self.transform(image=image)
#             image  = data['image']
#             image = np.transpose(image, (2, 0, 1))
#         return torch.tensor(image)
    
    
    
class ContrailDataset:
    def __init__(self, df, transform=None):
        self.df = df  
        self.images = df['image']
        self.labels = df['label']
        self.transform =transform
        
        
    def __len__(self):
        return len(self.images)
        
    def __getitem__(self, idx):
        image = np.load("../../input/" + self.images[idx]).astype(float)   
        label = np.load("../../input/" + self.labels[idx]).astype(float)
        
        
        # label_cls = 1 if label.sum() > 0 else 0
        if self.transform :
            data = self.transform(image=image, mask=label)
            image  = data['image']
            label  = data['mask']
            image = np.transpose(image, (2, 0, 1))
            label = np.transpose(label, (2, 0, 1))    
            
        return torch.tensor(image), torch.tensor(label)

In [8]:
###TImmunetplusplus model Nirjhar

n_blocks = 4

class TimmSegModel(nn.Module):
    def __init__(self, backbone, cfg, segtype='unet', pretrained=True):
        super(TimmSegModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 3, stride=1, padding=1, bias=False)
        self.conv2 = nn.Conv2d(6, 12, 3, stride=1, padding=1, bias=False)
        self.conv3 = nn.Conv2d(12, 36, 3, stride=1, padding=1, bias=False)
        self.mybn1 = nn.BatchNorm2d(6)
        self.mybn2 = nn.BatchNorm2d(12)
        self.mybn3 = nn.BatchNorm2d(36)     
        self.encoder = timm.create_model(
            backbone,
            in_chans=3,
            features_only=True,
            drop_rate=0.8,
            drop_path_rate=0.5,
            pretrained=False
        )
        self.encoder.conv_stem=nn.Conv2d(6, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)

        self.encoder.blocks[5] = nn.Identity()
        self.encoder.blocks[6] = nn.Sequential(
            nn.Conv2d(self.encoder.blocks[4][2].conv_pwl.out_channels, 320, 1),
            nn.BatchNorm2d(320),
            nn.ReLU6(),
        )
        tr = torch.randn(1,6,64,64)
        g = self.encoder(tr)
        encoder_channels = [1] + [_.shape[1] for _ in g]
        decoder_channels = [256, 128, 64, 32, 16]
        if segtype == 'unet':
            self.decoder = smp.decoders.unetplusplus.decoder.UnetPlusPlusDecoder(
                encoder_channels=encoder_channels[:n_blocks+1],
                decoder_channels=decoder_channels[:n_blocks],
                n_blocks=n_blocks,
            )

        self.segmentation_head = nn.Conv2d(
            decoder_channels[n_blocks-1],
            cfg["num_classes"], 
            kernel_size=(3, 3),
            stride=(1, 1), 
            padding=(1, 1)
        )

    def forward(self,x):
        x = F.relu6(self.mybn1(self.conv1(x)))
        global_features = [0] + self.encoder(x)[:n_blocks]
        seg_features = self.decoder(*global_features)
        seg_features = self.segmentation_head(seg_features)
        return seg_features
    

#path='./exp/baselinev2/qishentrialv2_tf_efficientnet_b7_ns_last_epoch-00.bin'

def build_model_timmunetplus(backbone, cfg):
    model = TimmSegModel(backbone, cfg)
    return model

def load_model_timmunetplus(path, backbone, cfg):
    model = build_model_timmunetplus(backbone, cfg)
    model.load_state_dict(torch.load(path))
    return model


#### Model 2 Nirjhar

n_blocks =4
class TimmSegModel2(nn.Module):
    def __init__(self, backbone, segtype='unet', pretrained=True):
        super(TimmSegModel2, self).__init__()

        self.encoder = timm.create_model(
            backbone,
            in_chans=3,
            features_only=True,
            drop_rate=0.5,
            pretrained=False
        )
        g = self.encoder(torch.rand(1, 3, 128, 128))
        encoder_channels = [1] + [_.shape[1] for _ in g]
        decoder_channels = [256, 128, 64, 32, 16]
        if segtype == 'unet':
            self.decoder = smp.decoders.unetplusplus.decoder.UnetPlusPlusDecoder(
                encoder_channels=encoder_channels[:n_blocks+1],
                decoder_channels=decoder_channels[:n_blocks],
                n_blocks=n_blocks,
            )

        self.segmentation_head = nn.Sequential(
            nn.Conv2d(decoder_channels[n_blocks-1], 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.UpsamplingBilinear2d(scale_factor=1))

    def forward(self,x):
        global_features = [0] + self.encoder(x)[:n_blocks]
        seg_features = self.decoder(*global_features)
        seg_features = self.segmentation_head(seg_features)
        return seg_features
    
#path='./exp/baselinev2/qishentrialv2_tf_efficientnet_b7_ns_last_epoch-00.bin'


def build_model(backbone):
    model = TimmSegModel2(backbone, segtype='unet')
    return model

def load_model(path,backbone):
    model = build_model(backbone)
    model.load_state_dict(torch.load(path))
    return model




## Rohit Model 
class Net(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.cfg = cfg
        
        self.model = smp.Unet(
            encoder_name=cfg["backbone"],     
            encoder_weights=None,   
            in_channels=3,  
            classes=cfg["num_classes"],
            activation=None
        )
        
    
    def forward(self, inputs):
        mask = self.model(inputs)
        return mask

In [9]:
def dice_coef(y_true, y_pred, thr=0.5, epsilon=1e-6):
    y_true = y_true.to(torch.float32)
    y_pred = (y_pred>thr).to(torch.float32)
    inter = (y_true*y_pred).sum()
    den = y_true.sum() + y_pred.sum()
    dice = ((2*inter+epsilon)/(den+epsilon)).mean()
    
    return dice


def get_transform(img_size):
    transform = A.Compose([
        A.Resize(*img_size, interpolation=cv2.INTER_NEAREST),
    ], p=1.0)
    return transform

In [10]:
val_df =pd.read_csv("../../input/val_df_filled.csv")
train_df =pd.read_csv("../../input/train_folds.csv")

# val_df = val_df.loc[val_df['class'] == 1].reset_index(drop=True)
# val_df = val_df.loc[val_df['class'] == 0].reset_index(drop=True)

final_preds = []

In [11]:

# for idx, cfg in enumerate(CFGS):
# #     if idx <= 6:
# #         continue
    
# #     val_df = pd.read_csv("../../input/val_df_filled.csv")
#     val_df = train_df.loc[train_df.fold == idx]
    
    
#     print(cfg)
#     val_transform = get_transform(cfg['img_size'])
#     valid_dataset = ContrailDataset(val_df, transform=val_transform)  

#     valid_loader = DataLoader(
#         valid_dataset, 
#         batch_size = 32, #32, 
#         shuffle = False, 
#         num_workers = 2, 
#         pin_memory = True, 
#         drop_last = False
#     )

    

#     if ensemple_type == "mine":
#         model = Net(cfg)    
#         model = torch.nn.DataParallel(model).cuda()
#         model.load_state_dict(torch.load(cfg['model_pth'], map_location=torch.device('cpu'))['model'])
#     else:
#         if idx <= 1:
#             model = load_model_timmunetplus(cfg['model_pth'], cfg['backbone'], cfg)
#         elif idx == 2:
#             model = load_model(cfg['model_pth'], cfg['backbone'])

#         else:
#             model = Net(cfg)    
#             model = torch.nn.DataParallel(model).cuda()
#             model.load_state_dict(torch.load(cfg['model_pth'], map_location=torch.device('cpu'))['model'])

#     if cfg['tta']:
#         if ensemple_type == "mine":
#             model = tta.SegmentationTTAWrapper(model, tta.aliases.flip_transform(), merge_mode='mean')
#         else:
#             if idx <= 2:
#                 model = tta.SegmentationTTAWrapper(model, tta.aliases.hflip_transform(), merge_mode='mean')
#             else:
#                 model = tta.SegmentationTTAWrapper(model, tta.aliases.flip_transform(), merge_mode='mean')

    
    
    
    
#     model.to(device)
#     model.eval()
    
#     preds = []
#     masks_ = []
    
#     for index, (images, masks) in enumerate(tqdm(valid_loader)):  
#         images  = images.to(device, dtype=torch.float)
#         masks  = masks.to(device, dtype=torch.float)
#         if cfg['img_size'][0] != 256:
#             masks = torch.nn.functional.interpolate(masks, size=256, mode='nearest') 
            
#         masks_.append(torch.squeeze(masks, dim=1))

#         with torch.inference_mode():
#             images = torch.nn.functional.interpolate(images,size=cfg['img_size'][0], mode='nearest')
#             pred = model(images)                     
#             pred = torch.nn.functional.interpolate(pred, size=256, mode='nearest') 

#             preds.append(torch.squeeze(pred, dim=1))

            
               
            
#     model_masks = torch.cat(masks_, dim=0)
#     model_preds = torch.cat(preds, dim=0)
    
#     model_masks = torch.flatten(model_masks, start_dim=0, end_dim=1)
#     model_preds = torch.flatten(model_preds, start_dim=0, end_dim=1)  
    
    
# #     torch.save(model_preds, f"../../output/final_preds/{cfg['call_sign']}.pt")
    
    
# #     if idx == 0:
# #         torch.save(model_masks, f'../../output/final_preds/val_masks.pt')

        
        
#     best_threshold = 0.0
#     best_dice_score = 0.0
#     for threshold in [i / 100 for i in range(101)] :
#         score = dice_coef(model_masks, model_preds.sigmoid(), thr=threshold).cpu().detach().numpy() 
#         if score > best_dice_score:
#             best_dice_score = score
#             best_threshold = threshold
    
        
#     print(best_dice_score, best_threshold)
#     final_preds.append(model_preds)
    
#     del model
#     torch.cuda.empty_cache()
#     gc.collect()

In [12]:

final_masks = []
final_preds = []

for idx, cfg in enumerate(CFGS):
    val_df = train_df.loc[train_df.fold == idx].reset_index(drop=True) 
#     val_df =pd.read_csv("../../input/val_df_filled.csv")

#     val_df = val_df.loc[val_df['class'] == 1]

    
    val_transform = get_transform(cfg['img_size'])
    valid_dataset = ContrailDataset(val_df, transform=val_transform)  

    valid_loader = DataLoader(
        valid_dataset, 
        batch_size = 32, #32, 
        shuffle = False, 
        num_workers = 2, 
        pin_memory = True, 
        drop_last = False
    )

    

    if ensemple_type == "mine":
        model = Net(cfg)    
        model = torch.nn.DataParallel(model).cuda()
        model.load_state_dict(torch.load(cfg['model_pth'], map_location=torch.device('cpu'))['model'])
    else:
        if idx <= 1:
            model = load_model_timmunetplus(cfg['model_pth'], cfg['backbone'], cfg)
        elif idx == 2:
            model = load_model(cfg['model_pth'], cfg['backbone'])

        else:
            model = Net(cfg)    
            model = torch.nn.DataParallel(model).cuda()
            model.load_state_dict(torch.load(cfg['model_pth'], map_location=torch.device('cpu'))['model'])

    if cfg['tta']:
        if ensemple_type == "mine":
            model = tta.SegmentationTTAWrapper(model, tta.aliases.flip_transform(), merge_mode='mean')
        else:
            if idx <= 2:
                model = tta.SegmentationTTAWrapper(model, tta.aliases.hflip_transform(), merge_mode='mean')
            else:
                model = tta.SegmentationTTAWrapper(model, tta.aliases.flip_transform(), merge_mode='mean')

    
    preds = []
    masks_ = []
    model.to(device)
    model.eval()
    
    for index, (images, masks) in enumerate(tqdm(valid_loader)):  
        images  = images.to(device, dtype=torch.float)
        masks  = masks.to(device, dtype=torch.float)
        if cfg['img_size'][0] != 256:
            masks = torch.nn.functional.interpolate(masks, size=256, mode='nearest') 
            
        masks_.append(torch.squeeze(masks, dim=1))

        with torch.inference_mode():
            images = torch.nn.functional.interpolate(images,size=cfg['img_size'][0], mode='nearest')
            pred = model(images)                     
            pred = torch.nn.functional.interpolate(pred.sigmoid(), size=256, mode='nearest') 
            preds.append(torch.squeeze(pred, dim=1))

                  
    del model, pred, images, masks
    torch.cuda.empty_cache()
    gc.collect()
            
    model_masks = torch.cat(masks_, dim=0)
    model_preds = torch.cat(preds, dim=0)    
    
#     model_masks = torch.flatten(model_masks, start_dim=0, end_dim=1)
#     model_preds = torch.flatten(model_preds, start_dim=0, end_dim=1)  
    
    final_preds.append(model_preds)
    final_masks.append(model_masks)
    
#     #####
#     best_threshold = 0.0
#     best_dice_score = 0.0
#     for threshold in [i / 100 for i in range(101)] :
#         score = dice_coef(model_masks, model_preds, thr=threshold).cpu().detach().numpy() 
#         if score > best_dice_score:
#             best_dice_score = score
#             best_threshold = threshold


#     print(best_dice_score, best_threshold)
#     final_preds.append(model_preds)
    
    
    
#     torch.save(model_preds, f"../../output/final_preds/{cfg['call_sign']}.pt")
#     if idx == 0:
#         torch.save(model_preds, f"../../output/final_preds/{cfg['call_sign']}_{idx}.pt")



# best_threshold = 0.0
# best_dice_score = 0.0
# for threshold in [i / 100 for i in range(101)] :
#     score = dice_coef(model_masks, model_preds.sigmoid(), thr=threshold).cpu().detach().numpy() 
#     if score > best_dice_score:
#         best_dice_score = score
#         best_threshold = threshold


# print(best_dice_score, best_threshold)
# final_preds.append(model_preds)

# del model
# torch.cuda.empty_cache()
# gc.collect()

  0%|          | 0/140 [00:00<?, ?it/s]

  0%|          | 0/140 [00:00<?, ?it/s]

  0%|          | 0/140 [00:00<?, ?it/s]

In [13]:
# final_preds[0].shape, final_preds[1].shape, final_preds[2].shape, final_preds[3].shape, final_preds[4].shape

final_preds = torch.cat(final_preds, dim=0)
final_preds = torch.flatten(final_preds, start_dim=0, end_dim=1)  


final_masks = torch.cat(final_masks, dim=0)
final_masks = torch.flatten(final_masks, start_dim=0, end_dim=1)  


In [14]:
final_masks.shape, final_preds.shape

(torch.Size([3438592, 256]), torch.Size([3438592, 256]))

In [16]:
# score = dice_coef(final_masks, final_preds, thr=0.4).cpu().detach().numpy() 

In [17]:
# final_preds = torch.stack(final_preds).mean(dim=0)

best_threshold = 0.0
best_dice_score = 0.0
for threshold in [i / 100 for i in range(101)] :
    score = dice_coef(final_masks, final_preds, thr=threshold).cpu().detach().numpy() 
    if score > best_dice_score:
        best_dice_score = score
        best_threshold = threshold


print(best_dice_score, best_threshold)
# final_preds.append(model_preds)

# del model
# torch.cuda.empty_cache()
# gc.collect()

0.6910231 0.39


In [21]:
# final_preds = torch.stack(final_preds).mean(dim=0)

In [22]:
# final_preds = [
#     torch.load(f"../../output/final_preds/roh_1_00.pt"),
#     torch.load(f"../../output/final_preds/roh_1_01.pt"),
#     torch.load(f"../../output/final_preds/roh_1_02.pt"),
#     torch.load(f"../../output/final_preds/roh_1_03.pt")
# ]

# final_preds = torch.stack(final_preds).mean(dim=0)

In [23]:
masks_ = [torch.load(f"../../output/final_preds/roh_1_00_0.pt")]

preds = [
    torch.load(f"../../output/final_preds/roh_1_00.pt"),
    torch.load(f"../../output/final_preds/roh_1_01.pt"),
    torch.load(f"../../output/final_preds/roh_1_02.pt"),
    torch.load(f"../../output/final_preds/roh_1_03.pt"),
    torch.load(f"../../output/final_preds/roh_1_04.pt"),
]


final_preds = torch.stack(preds).mean(dim=0)

RuntimeError: stack expects each tensor to be equal size, but got [1856, 256, 256] at entry 0 and [3712, 256, 256] at entry 1

In [11]:
model_masks = torch.cat(masks_, dim=0)
model_preds = torch.cat(preds[:2], dim=0)

model_masks = torch.flatten(model_masks, start_dim=0, end_dim=1)
model_preds = torch.flatten(model_preds, start_dim=0, end_dim=1)  



best_threshold = 0.0
best_dice_score = 0.0
for threshold in [i / 100 for i in range(101)] :
    score = dice_coef(model_masks, model_preds, thr=threshold).cpu().detach().numpy() 
    print(score, threshold)
    if score > best_dice_score:
        best_dice_score = score
        best_threshold = threshold


print(best_dice_score, best_threshold)
final_preds.append(model_preds)

del model
torch.cuda.empty_cache()
gc.collect()

RuntimeError: The size of tensor a (1146368) must match the size of tensor b (3438848) at non-singleton dimension 0

In [18]:
# model_preds.sigmoid()

OutOfMemoryError: CUDA out of memory. Tried to allocate 5.47 GiB (GPU 0; 23.69 GiB total capacity; 21.86 GiB already allocated; 240.19 MiB free; 22.20 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [14]:
# model_masks = torch.load("../../output/final_preds/val_masks.pt")

# model_names = ["nir_01", "nir_02", "nir_03", "roh_01", "roh_02", "roh_03", "roh_04", "roh_05", "roh_06", "roh_07",  ]
# old_preds = []
# for name in tqdm(model_names, total=len(model_names)):
#     old_preds.append(torch.load(f"../../output/final_preds/{name}.pt"))

# if len(final_preds) > 0:
#     final_preds.extend(old_preds)
# else:
#     final_preds = old_preds

In [ ]:
# score = dice_coef(model_masks, final_preds[0], thr=0.5).cpu().detach().numpy() # get_score(labels, preds) 
# score

# 0.65458226 0.83
# 0.6572782 0.1

len(final_preds)

In [ ]:
def boost_dice(params, TH=None):    
    for index, val in enumerate(params.keys()):
        if index == 0:            
            preds = params[val]*final_preds[0]
        else:
            preds += params[val]*final_preds[index]
    
    
    param_sum = 0
    for key, val in params.items():
        param_sum += val

    preds = preds/param_sum
    
    
    if TH:
        best_threshold = TH
        preds1 = (preds.sigmoid()).double()            
        best_score = dice_coef(model_masks, preds1, thr=best_threshold).cpu().detach().numpy() # get_score(labels, preds) 
    else:
        best_threshold = 0.0
        best_score = 0.0
        for threshold in [i / 100 for i in range(101)] :    
            preds1 = (preds.sigmoid()>threshold).double()            
            score = dice_coef(model_masks, preds1, thr=threshold).cpu().detach().numpy() # get_score(labels, preds) 
            if score > best_score:
                best_score = score
                best_threshold = threshold
            
#     best_score = score
#     best_threshold = threshold
    return best_score, best_threshold

In [ ]:
def objective(trial):
    TH = None
    
    params = {}    
    for i in range(len(final_preds)):
        params[f"w{i+1}"] = trial.suggest_float(f'w{i+1}', 0, 1) 
        
        
    score, best_threshold  = boost_dice(params, TH=TH)
    params['threshold'] = best_threshold
    if not TH:
        print(params)
    return score

study = optuna.create_study(direction='maximize')
# study = optuna.create_study(direction='minimize')

study.optimize(objective, n_trials=1500)

best_params = study.best_params
best_params

In [20]:

# [I 2023-06-16 19:56:17,542] | TH:0.11 | Trial 2772 finished with value: 0.685997724533081 and parameters: {'w1': 0.2728695549458137, 'w2': 0.32228366602907965, 'w3': 0.5198602860436105, 'w4': 0.3645111903670712, 'w5': 0.9222454111482654, 'w6': 0.513409920082969, 'w7': 0.8675021234700891, 'w8': 0.0722146925058941, 'w9': 0.09988548514924554, 'w10': 0.04023375737773327}. Best is trial 2772 with value: 0.685997724533081.

#  th:0.10 | Best is trial 665 with value: 0.686022162437439.
# {'w1': 0.07249514832914417,
#  'w2': 0.3044527524365949,
#  'w3': 0.46339624897419424,
#  'w4': 0.30916961792390085,
#  'w5': 0.9820263596044505,
#  'w6': 0.13629275856955803,
#  'w7': 0.9819243920033219,
#  'w8': 0.10786742006569568,
#  'w9': 0.017963370847088496,
#  'w10': 0.2269773365337288}




# th:0.09 Trial 599 finished with value: 0.6861019730567932 and parameters: {'w1': 0.08411983424429813, 'w2': 0.2397144985551996, 'w3': 0.565597681400598, 'w4': 0.27365134829998194, 'w5': 0.8519007002554051, 'w6': 0.0034467714906157854, 'w7': 0.9632070992690323, 'w8': 0.036223969967080535, 'w9': 0.1408041912398369, 'w10': 0.0808046168343181}. Best is trial 599 with value: 0.6861019730567932.






# Best is trial 979 th:0.08 | with value: 0.6860898733139038.
# {'w1': 0.04219070925378117,
#  'w2': 0.2814610861485835,
#  'w3': 0.45306132583945036,
#  'w4': 0.08300507686599938,
#  'w5': 0.9689922153717234,
#  'w6': 0.5832699926479489,
#  'w7': 0.8380073187640669,
#  'w8': 0.00027139503974531664,
#  'w9': 0.08843006287891712,
#  'w10': 0.2639847795838674}




# 0.6835659742355347 and parameters: {'w1': 0.012511663696378001, 'w2': 0.7308678705489249, 'w3': 0.7309363859834449, 'w4': 0.8998292052691779, 'w5': 0.9377896156028775}. Best is trial 183 with value: 0.6835659742355347.
# TH: 0.1

# pl3_round2_s2 included at last
# Trial 177 finished with value: 0.6850342154502869 and parameters: 
#{'w1': 0.1976000599839957, 'w2': 0.4126970697935236, 'w3': 0.0010400328806771247, 'w4': 0.13358923329787326, 'w5': 0.6293367385421876, 'w6': 0.8157294847307331}. Best is trial 1286 with value: 0.6850342154502869.
#TH:0.19


# 0.6850342154502869
# {'w1': 0.1976000599839957,
#  'w2': 0.4126970697935236,
#  'w3': 0.0010400328806771247,
#  'w4': 0.13358923329787326,
#  'w5': 0.6293367385421876,
#  'w6': 0.8157294847307331}



#  0.6864011883735657 and parameters: 
# {'w1': 0.0037528376744131176, 'w2': 0.1968372043477828, 'w3': 0.324173594919189,
#  'w4': 0.08069037545373889, 'w5': 0.666119680919894, 'w6': 0.23792233503063126, 'w7': 0.6474738231385013, 
#  }
#. Best is trial 301 with value: 0.6864011883735657.
# TH: 0.11



#th: 0.2
# {'w1': 0.9118427143017789, 'w2': 0.07722806128805348, 'w3': 0.3483576258941602}


# 0.6802493929862976
# {'w1': 0.901456237984701,
#  'w2': 0.03333500127663481,
#  'w3': 0.1012744451767772,
#  'w4': 0.3315684160625839}


# 0.6802645325660706
# {'w1': 0.9423043059931125,
#  'w2': 0.12999481950972483,
#  'w3': 0.34096541343299724}




# 0.6753096580505371
# {'w1': 0.8191544364616984,
#  'w2': 0.2172873170900641,
#  'w3': 0.9872223348463418,
#  'w4': 0.34763898831692425,
#  'w5': 0.6106201134139329}


In [ ]:
best_params

In [ ]:
# {'w1': 0.19193466652838265, 'w2': 0.0010692529855302647, 'w3': 0.19479657217496682, 'w4': 0.7062431810854415, 'threshold': 0.39}


In [ ]:
#  0.66671222448349 and parameters:
# {'w1': 0.7800660379809229, 'w2': 0.5851307264271756, 'w3': 0.5728701966485974}
# threshold: 0.32


In [204]:
import pretrainedmodels

def conv3x3(in_channel, out_channel): #not change resolusion
    return nn.Conv2d(in_channel,out_channel,
                      kernel_size=3,stride=1,padding=1,dilation=1,bias=False)

def conv1x1(in_channel, out_channel): #not change resolution
    return nn.Conv2d(in_channel,out_channel,
                      kernel_size=1,stride=1,padding=0,dilation=1,bias=False)

def init_weight(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        #nn.init.xavier_uniform_(m.weight, gain=1)
        #nn.init.xavier_normal_(m.weight, gain=1)
        #nn.init.kaiming_uniform_(m.weight, mode='fan_in', nonlinearity='relu')
        nn.init.kaiming_normal_(m.weight, mode='fan_in', nonlinearity='relu')
        #nn.init.orthogonal_(m.weight, gain=1)
        if m.bias is not None:
            m.bias.data.zero_()
    elif classname.find('Batch') != -1:
        m.weight.data.normal_(1,0.02)
        m.bias.data.zero_()
    elif classname.find('Linear') != -1:
        nn.init.orthogonal_(m.weight, gain=1)
        if m.bias is not None:
            m.bias.data.zero_()
    elif classname.find('Embedding') != -1:
        nn.init.orthogonal_(m.weight, gain=1)
    

class ChannelAttentionModule(nn.Module):
    def __init__(self, in_channel, reduction):
        super().__init__()
        self.global_maxpool = nn.AdaptiveMaxPool2d(1)
        self.global_avgpool = nn.AdaptiveAvgPool2d(1) 
        self.fc = nn.Sequential(
            conv1x1(in_channel, in_channel//reduction).apply(init_weight),
            nn.ReLU(True),
            conv1x1(in_channel//reduction, in_channel).apply(init_weight)
        )
        
    def forward(self, inputs):
        x1 = self.global_maxpool(inputs)
        x2 = self.global_avgpool(inputs)
        x1 = self.fc(x1)
        x2 = self.fc(x2)
        x  = torch.sigmoid(x1 + x2)
        return x
    
    
class SpatialAttentionModule(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv3x3 = conv3x3(2,1).apply(init_weight)
        
    def forward(self, inputs):
        x1,_ = torch.max(inputs, dim=1, keepdim=True)
        x2 = torch.mean(inputs, dim=1, keepdim=True)
        x  = torch.cat([x1,x2], dim=1)
        x  = self.conv3x3(x)
        x  = torch.sigmoid(x)
        return x
    
    
class CBAM(nn.Module):
    def __init__(self, in_channel, reduction):
        super().__init__()
        self.channel_attention = ChannelAttentionModule(in_channel, reduction)
        self.spatial_attention = SpatialAttentionModule()
        
    def forward(self, inputs):
        x = inputs * self.channel_attention(inputs)
        x = x * self.spatial_attention(x)
        return x
    
    
class CenterBlock(nn.Module):
    def __init__(self, in_channel, out_channel):
        super().__init__()
        self.conv = conv3x3(in_channel, out_channel).apply(init_weight)
        
    def forward(self, inputs):
        x = self.conv(inputs)
        return x


class DecodeBlock(nn.Module):
    def __init__(self, in_channel, out_channel, upsample):
        super().__init__()
        self.bn1 = nn.BatchNorm2d(in_channel).apply(init_weight)
        self.upsample = nn.Sequential()
        if upsample:
            self.upsample.add_module('upsample',nn.Upsample(scale_factor=2, mode='nearest'))
        self.conv3x3_1 = conv3x3(in_channel, in_channel).apply(init_weight)
        self.bn2 = nn.BatchNorm2d(in_channel).apply(init_weight)
        self.conv3x3_2 = conv3x3(in_channel, out_channel).apply(init_weight)
        self.cbam = CBAM(out_channel, reduction=16)
        self.conv1x1   = conv1x1(in_channel, out_channel).apply(init_weight)
        
    def forward(self, inputs):
        x  = F.relu(self.bn1(inputs))
        x  = self.upsample(x)
        x  = self.conv3x3_1(x)
        x  = self.conv3x3_2(F.relu(self.bn2(x)))
        x  = self.cbam(x)
        x += self.conv1x1(self.upsample(inputs)) #shortcut
        return x
 
            
#U-Net SeResNext101 + CBAM + hypercolumns + deepsupervision
class UNET_SERESNEXT101(nn.Module):
    def __init__(self, resolution, deepsupervision, clfhead, clf_threshold, load_weights=True):
        super().__init__()
        h,w = resolution
        self.deepsupervision = deepsupervision
        self.clfhead = clfhead
        self.clf_threshold = clf_threshold
        
        #encoder
        model_name = 'se_resnext101_32x4d'
        if load_weights:
            seresnext101 = pretrainedmodels.__dict__[model_name](pretrained='imagenet')
        else:
            seresnext101 = pretrainedmodels.__dict__[model_name](pretrained=None)
        
        self.encoder0 = nn.Sequential(
            seresnext101.layer0.conv1, #(*,3,h,w)->(*,64,h/2,w/2)
            seresnext101.layer0.bn1,
            seresnext101.layer0.relu1,
        )
        self.encoder1 = nn.Sequential(
            seresnext101.layer0.pool, #->(*,64,h/4,w/4)
            seresnext101.layer1 #->(*,256,h/4,w/4)
        )
        self.encoder2 = seresnext101.layer2 #->(*,512,h/8,w/8)
        self.encoder3 = seresnext101.layer3 #->(*,1024,h/16,w/16)
        self.encoder4 = seresnext101.layer4 #->(*,2048,h/32,w/32)
        
        #center
        self.center  = CenterBlock(2048,512) #->(*,512,h/32,w/32)
        
        #decoder
        self.decoder4 = DecodeBlock(512+2048,64, upsample=True) #->(*,64,h/16,w/16)
        self.decoder3 = DecodeBlock(64+1024,64, upsample=True) #->(*,64,h/8,w/8)
        self.decoder2 = DecodeBlock(64+512,64,  upsample=True) #->(*,64,h/4,w/4) 
        self.decoder1 = DecodeBlock(64+256,64,   upsample=True) #->(*,64,h/2,w/2) 
        self.decoder0 = DecodeBlock(64,64, upsample=True) #->(*,64,h,w) 
        
        #upsample
        self.upsample4 = nn.Upsample(scale_factor=16, mode='bilinear', align_corners=True)
        self.upsample3 = nn.Upsample(scale_factor=8, mode='bilinear', align_corners=True)
        self.upsample2 = nn.Upsample(scale_factor=4, mode='bilinear', align_corners=True)
        self.upsample1 = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        
        #deep supervision
        self.deep4 = conv1x1(64,1).apply(init_weight)
        self.deep3 = conv1x1(64,1).apply(init_weight)
        self.deep2 = conv1x1(64,1).apply(init_weight)
        self.deep1 = conv1x1(64,1).apply(init_weight)
        
        #final conv
        self.final_conv = nn.Sequential(
            conv3x3(320,64).apply(init_weight),
            nn.ELU(True),
            conv1x1(64,1).apply(init_weight)
        )
        
        #clf head
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.clf = nn.Sequential(
            nn.BatchNorm1d(2048).apply(init_weight),
            nn.Linear(2048,512).apply(init_weight),
            nn.ELU(True),
            nn.BatchNorm1d(512).apply(init_weight),
            nn.Linear(512,1).apply(init_weight)
        )
        
        
    def forward(self, inputs):
        #encoder
        x0 = self.encoder0(inputs) #->(*,64,h/2,w/2)
        x1 = self.encoder1(x0) #->(*,256,h/4,w/4)
        x2 = self.encoder2(x1) #->(*,512,h/8,w/8)
        x3 = self.encoder3(x2) #->(*,1024,h/16,w/16)
        x4 = self.encoder4(x3) #->(*,2048,h/32,w/32)
        
        #clf head
        logits_clf = self.clf(self.avgpool(x4).squeeze(-1).squeeze(-1)) #->(*,1)
        if (not self.training) & (self.clf_threshold is not None):
            if (torch.sigmoid(logits_clf)>self.clf_threshold).sum().item()==0:
                bs,_,h,w = inputs.shape
                logits = torch.zeros((bs,1,h,w))
                if self.clfhead:
                    if self.deepsupervision:
                        return logits,_,_
                    else:
                        return logits,_
                else:
                    if self.deepsupervision:
                        return logits,_
                    else:
                        return logits
        
        #center
        y5 = self.center(x4) #->(*,320,h/32,w/32)
        
        #decoder
        y4 = self.decoder4(torch.cat([x4,y5], dim=1)) #->(*,64,h/16,w/16)
        y3 = self.decoder3(torch.cat([x3,y4], dim=1)) #->(*,64,h/8,w/8)
        y2 = self.decoder2(torch.cat([x2,y3], dim=1)) #->(*,64,h/4,w/4)
        y1 = self.decoder1(torch.cat([x1,y2], dim=1)) #->(*,64,h/2,w/2) 
        y0 = self.decoder0(y1) #->(*,64,h,w)
        
        #hypercolumns
        y4 = self.upsample4(y4) #->(*,64,h,w)
        y3 = self.upsample3(y3) #->(*,64,h,w)
        y2 = self.upsample2(y2) #->(*,64,h,w)
        y1 = self.upsample1(y1) #->(*,64,h,w)
        hypercol = torch.cat([y0,y1,y2,y3,y4], dim=1)
        
        #final conv
        logits = self.final_conv(hypercol) #->(*,1,h,w)
        
        if self.clfhead:
            if self.deepsupervision:
                s4 = self.deep4(y4)
                s3 = self.deep3(y3)
                s2 = self.deep2(y2)
                s1 = self.deep1(y1)
                logits_deeps = [s4,s3,s2,s1]
                return logits, logits_deeps, logits_clf
            else:
                return logits, logits_clf
        else:
            if self.deepsupervision:
                s4 = self.deep4(y4)
                s3 = self.deep3(y3)
                s2 = self.deep2(y2)
                s1 = self.deep1(y1)
                logits_deeps = [s4,s3,s2,s1]
                return logits, logits_deeps
            else:
                return logits
            
            


In [205]:
def build_model(model_name, resolution, deepsupervision, clfhead, clf_threshold, load_weights):
    if model_name=='seresnext101':
        model = UNET_SERESNEXT101(resolution, deepsupervision, clfhead, clf_threshold, load_weights)
    return model

In [219]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context


model = build_model("seresnext101", (256,256), False, False, 0.5, load_weights=True )
# model.cuda()


In [220]:
# model()

In [222]:
a = torch.ones(64, 3, 256, 256)
out = model(a)

In [224]:
out.shape

torch.Size([64, 1, 256, 256])